In [1]:
import h5py
import numpy as np 
with h5py.File('images_training.h5','r') as H:
    data = np.copy(H['data']) 
with h5py.File('labels_training.h5','r') as H:
    label = np.copy(H['label'])

/Users/geyang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
data.shape

(30000, 28, 28)

In [3]:
label[-1]

2

In [4]:
m=data.shape[1]*data.shape[2]
data1 = data.reshape(30000,m)
del data

In [5]:
data1.shape

(30000, 784)

In [6]:
target = set()
for i in label:
    target.add(i)
d = len(list(target))
d

10

In [7]:
# ovr, label "1" vs. not label "1" 
label1 = np.array([])
for i in range(d):
    label_tmp=np.copy(label)
    for tmp in range(len(label_tmp)):
        if label_tmp[tmp] != i:
            label_tmp[tmp] = 0
        else:
            label_tmp[tmp] = 1
    label_tmp= label_tmp.reshape(-1,1)
    if i == 0:
        label1 = np.copy(label_tmp)
    else:
        label1 = np.append(label1, label_tmp, axis=1)
del label_tmp

In [8]:
label1.shape


(30000, 10)

In [9]:
label[-1]

2

In [10]:
label1

array([[1, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]], dtype=uint8)

In [11]:
#used scikit-learn library, need to implemenet ourselve later
from sklearn.preprocessing import normalize
data_normalize = normalize(data1)
del data1

In [12]:
data_normalize.shape

(30000, 784)

In [13]:
class PCA():
    def calculate_covariance_matrix(self, X, Y=None):
        # 计算协方差矩阵

        m = X.shape[0]
        X = X - np.mean(X, axis=0)
        Y = X if Y == None else Y - np.mean(Y, axis=0)
        return 1 / m * np.matmul(X.T, Y)

    def transform(self, X, n_components):
        # 设n=X.shape[1]，将n维数据降维成n_component维

        covariance_matrix = self.calculate_covariance_matrix(X)

        # 获取特征值，和特征向量
        eigenvalues, eigenvectors = np.linalg.eig(covariance_matrix)

        # 对特征向量排序，并取最大的前n_component组
        idx = eigenvalues.argsort()[::-1]
        eigenvectors = eigenvectors[:, idx]
        eigenvectors = eigenvectors[:, :n_components]

        # 转换
        return np.matmul(X, eigenvectors), eigenvectors


In [14]:
data_normalize, eigenvectors = PCA().transform(data_normalize, 200)

In [15]:
data_normalize.shape

(30000, 200)

In [16]:
#from tutorial
def logistic_loss_and_grad(weights, inputs, targets):

    #derivative of the loss function
    z = np.dot(inputs, weights)
    exp_z = np.exp(z)
    temp = -exp_z/(1 + exp_z) + targets
#     print(temp.shape, inputs.shape, (temp*inputs).shape)
    dloss = -np.sum(temp*inputs, axis=0)[:, np.newaxis]

    return dloss

In [17]:
import numpy.linalg as lng
# weights = np.random.random(data1.shape[1])[:, np.newaxis]
weights = np.array([])
eta = 0.03
# n_iter = 2000
n_iter = 2000
eps = 1e-10

for i in range(d):
    weight = np.random.random(data_normalize.shape[1])[:, np.newaxis]
    current_label = (label1[:,i]).reshape(-1,1)
    print(i)
    
    for epoch in range(n_iter):
        dloss = logistic_loss_and_grad(weight, data_normalize, current_label)
        weight_new = weight - dloss * eta
        diff = lng.norm(weight_new - weight)
        weight = weight_new
        if diff < eps:
            break
    if i == 0:
        weights = weight
    else:
        weights = np.append(weights, weight, axis = 1)

0
1
2
3
4
5
6
7
8
9


In [18]:
weights.shape

(200, 10)

In [19]:

with h5py.File('images_testing.h5','r') as H:
    data_testing = np.copy(H['data']) 
with h5py.File('labels_testing_2000.h5','r') as H:
    label_testing = np.copy(H['label'])

In [20]:
data_testing = np.delete(data_testing, np.s_[2000:5000], 0)

In [21]:
data_testing = data_testing.reshape(2000,-1)
data_testing = np.matmul(data_testing, eigenvectors)
print(data_testing.shape)
print(label_testing.shape)

(2000, 200)
(2000,)


In [22]:
calculation = data_testing.dot(weights)
calculation

array([[-160614.94249018,  -35981.40845681, -237369.79265982, ...,
        -151227.82948727,  -86078.02561989, -119845.5077393 ],
       [-355762.10142245,  -47121.27525448, -232143.98935492, ...,
        -464442.15657574, -112035.76728436, -470410.11469028],
       [-465308.43657988,  -40526.46958163, -379896.48591458, ...,
        -452757.67793719,  -60699.06598486, -467735.12799684],
       ...,
       [-377070.03619057,  -32023.85170745, -166511.43453529, ...,
        -420589.79369454,  -97229.47690385, -425911.1597071 ],
       [-239177.66436525,  -87337.00022889, -424057.78325061, ...,
        -160157.48732049, -168556.28765592,   31592.79178458],
       [ -46770.06016709,  -17648.5232588 ,  -80784.06887655, ...,
        -118122.04132521,  -29010.70597599,  -77926.84133331]])

In [23]:
result = np.argmax(calculation, axis=1)
result.shape

(2000,)

In [24]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
confusion_matrix(label_testing, result)

array([[ 63,  52,   0,  37,   0,   0,  13,   0,  13,   0],
       [  0, 186,   0,   5,   0,   0,   0,   0,   0,   0],
       [  0,  97,  77,   4,   7,   1,   3,   0,  21,   0],
       [  0,   6,   0, 182,   0,   0,   1,   0,   2,   0],
       [  0,  95,   5,  27,  59,   0,   9,   0,  17,   0],
       [  0,   1,   0,   1,   0, 190,   0,   8,   1,  13],
       [  4,  54,   9,  25,   8,   0,  81,   0,  19,   0],
       [  0,   0,   0,   0,   0,  12,   0, 179,   0,   7],
       [  0,   1,   0,   4,   1,   3,   1,   1, 208,   0],
       [  0,   0,   0,   0,   0,   3,   0,   3,   1, 180]])

In [25]:
print(accuracy_score(label_testing, result)*100, "%")

70.25 %
